In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

Tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'tuning')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
# Alternate palette for promoter panel
metadata_alt_dict = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'promoters').set_index('construct').to_dict('dict')
promoter_palette = metadata_alt_dict['color']
promoter_markers = metadata_alt_dict['markers']

Plasmid titration (`data2`)

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'plasmid_titration')

In [ ]:
greys = matplotlib.colors.ListedColormap(matplotlib.colormaps['Greys'](np.linspace(0.4,1,256)))
teals = sns.light_palette(base.colors['teal'], 4)[1:] + sns.dark_palette(base.colors['teal'], 4, reverse=True)[1:-1]
amounts = sorted([float(d) for d in data2['amount'].unique() if not np.isnan(d)])
teals_palette = dict(zip(amounts, teals))

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs
scatter_kwargs['jitter'] = 0.1
annotate_kwargs = base.annotate_kwargs

# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(base.figure_width['full'], 6.25))
fig_gridspec = matplotlib.gridspec.GridSpec(4, 6, figure=fig, wspace=0.4, hspace=0.4, 
                                            
                                            height_ratios=[2.75,1,0.5,2], width_ratios=[1]*6)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:2]),
    'B': fig.add_subfigure(fig_gridspec[0,2:4]),
    'C': fig.add_subfigure(fig_gridspec[1:3,:4]),
    'D': fig.add_subfigure(fig_gridspec[:3,4:]),
    'E': fig.add_subfigure(fig_gridspec[3,:4]),
    'F': fig.add_subfigure(fig_gridspec[2:,4:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_tuning'/'fig_tuning.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.52, bottom=0.45, top=1.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.3)))

biorep = 2
plot_df = quantiles[(((quantiles['miR'].isin(['miR.FF4','miR.FF5'])) & (quantiles['group']=='controller') & (quantiles['design']==1) & (quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (quantiles['group']=='base')) &
                         (quantiles['promoter']=='EF1a') & (quantiles['biorep']==biorep) & (quantiles['exp']!='exp90.4')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', ylim=(2e1,1e5), xlabel='marker')
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=base.font_sizes['smaller_size'])
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)

for ax in axes: ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.52, bottom=0.45, top=1.3, right=0.1)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,0.3)))

biorep = 2
plot_df = quantiles[(((quantiles['miR'].isin(['miRE.FF4','miRE.FF5'])) & (quantiles['group']=='controller') & (quantiles['design']==1) & (quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1'])))) &
                         (quantiles['promoter']=='EF1a') & (quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', ylim=(2e1,1e5), xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='construct', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.despine(ax=ax, bottom=True)
ax.set(xlabel='', ylim=axes[0].get_ylim(), ylabel='', yticklabels=[])
ax.get_xaxis().set_visible(False)

for ax in axes: ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.52, bottom=0.38, top=0.35, right=0.2)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.2))

miR_order = ['none','miR.FF4','miR.FF5','miRE.FF4','miRE.FF5']
plot_df = stats[(((stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['miR'].unique())-0.5)
pairs = [('OL','CL')]

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Output mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(1e2,2e4))
ax.minorticks_off()
annotator = Annotator(ax, pairs, data=plot_df, x='miR', y='output_gmean', hue='construct', palette=main_palette, **scatter_kwargs)
annotator.configure(**annotate_kwargs).apply_test()

# slope
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0.4,1.2), ylabel='',
       yticks=[0.5,0.75,1], yticklabels=['0.5','','1.0'])

for ax in axes:
    ax.set_xticklabels([l.get_text().replace('.','-\n') for l in ax.get_xticklabels()])
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

In [ ]:
# miR.FF4 promoters
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.8, top=1.4, right=0.15)
axes = subfig.subplots(2, 1, gridspec_kw=dict(hspace=0.5))

promoter_order = ['EF1a','CAG','EFS','hPGK']
plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) | (stats['group']=='base'))].copy()
plot_df['promoter'] = plot_df['promoter'].astype(pd.api.types.CategoricalDtype(categories=promoter_order, ordered=True))
plot_df.sort_values(['promoter','group','ts_kind'], inplace=True)

# remove outlier hPGK point
plot_df = plot_df[~((plot_df['promoter']=='hPGK') & (plot_df['exp']=='exp92') & (plot_df['biorep']==1))]

# shift xticks to add more space between promoter groups
buffer = 0.6
xlim = (-0.5, len(plot_df['construct'].unique())-0.5+buffer*(len(promoter_order)-1))
xtick_locs = np.concatenate([[x+i*(3+buffer) for x in range(3)] for i in range(len(promoter_order))])
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
scatter_kwargs2 = dict(s=4, jitter=0.1, linewidth=0.5, edgecolor='white', native_scale=True)

# define pairs for statistical test
for promoter in promoter_order:
    construct_base = plot_df[(plot_df['promoter']==promoter), ]
# TODO

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=promoter_palette,
                    legend=False, ax=ax, marker=promoter_markers[construct], **scatter_kwargs2)
ax.set(title='Output mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
annotator = Annotator(ax, pairs, data=plot_df, x='construct_loc', y='output_gmean', **scatter_kwargs)
annotator.configure(**annotate_kwargs).apply_test()

# slope
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=promoter_palette,
                    legend=False, ax=ax, marker=promoter_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])
ax.axhline(marker_baseline, color='black', ls=':')

for i,ax in enumerate(axes):
    # add shaded regions every other promoter
    span1 = (xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2, xtick_locs[5]+(xtick_locs[6]-xtick_locs[5])/2,)
    span2 = (xtick_locs[8]+(xtick_locs[9]-xtick_locs[8])/2, xtick_locs[11]+(1+buffer)/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.axvspan(*span2, color=base.get_light_color(base.colors['gray']), alpha=0.2,)

    # add promoter labels
    for j, prom in enumerate(promoter_order):
        yloc = -6
        ax.annotate(prom, (1+j*(3+buffer),yloc), xycoords=('data','axes points'), ha='center', va='top', fontsize=base.font_sizes['smaller_size'])

    ax.set(xticklabels=['']*len(ax.get_xticklabels()))
    ax.minorticks_off()
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.35, top=1.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.4))

plot_df = stats[(stats['group']=='controller') & (stats['miR']=='miRE.FF4') & (stats['design']==1) & (stats['promoter']=='EF1a')].copy()
xlim = (-0.5, len(plot_df['ts_num'].unique())-0.5)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_num', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Output mean', xlim=xlim, xlabel='# target sites', ylabel='', yscale='log',)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# slope
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_num', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='',  ylabel='', ylim=(0.3,1.25), yticks=[0.4,0.6,0.8,1,1.2])

for ax in axes: sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.52, bottom=0.35, top=0.85, right=0.13)
axes = subfig.subplots(1,3, gridspec_kw=dict(wspace=1.1))

plot_df = quantiles2[(quantiles2['group']=='base') & (quantiles2['exp']=='exp100.1')]
plot_df2 = quantiles2[(quantiles2['ts_kind']=='T') & (quantiles2['exp']=='exp100.1')]
plot_df3 = stats2[stats2['construct']!='AIO.039']

# line plot, base
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=greys, ax=ax,
             legend=False, dashes=False, style='construct', markers=main_markers, hue_norm=matplotlib.colors.LogNorm(),
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             markersize=4, markeredgecolor='white', markeredgewidth=0.5)
ax.set(xscale='log', yscale='log', xlabel='marker',  title='base gene',) #xlim=(2e2,1e4)) #ylim=(2e1,4e3),
marker_baseline = stats2.loc[(stats2['exp']=='exp100.1') & (stats2['construct']=='AIO.039'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
ax.yaxis.set_tick_params(which='minor', left=False)

# line plot, CL
ax = axes[1]
sns.lineplot(data=plot_df2, x='bin_marker_quantiles_median', y='output', hue='amount', palette=teals_palette, ax=ax,
             legend=False, dashes=False, style='construct', markers=main_markers, hue_norm=matplotlib.colors.LogNorm(),
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             markersize=4, markeredgecolor='white', markeredgewidth=0.5)
ax.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,4e3))
ax.set_title(label='CL circuit', color=base.colors['teal'])
marker_baseline = stats2.loc[(stats2['exp']=='exp100.1') & (stats2['construct']=='AIO.039'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
ax.minorticks_off()

# stats slope vs amount
ax = axes[2]
sns.lineplot(data=plot_df3, x='amount', y='slope', hue='construct', palette=main_palette, ax=ax,
             legend=False, dashes=False, errorbar=None)
sns.scatterplot(data=plot_df3, x='amount', y='slope', hue='construct', palette=main_palette,
                style='construct', markers=main_markers, legend=False, ax=ax, s=(4**2), edgecolor='white', linewidth=0.5)
ax.set(ylim=(0,1.2), xlabel='rel. plasmid amount', xticks=[0.5**i for i in range(5)]+[0], xticklabels=['1.0','0.5','','','',''], xlim=(0,1.1))
sns.despine()

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_tuning.pdf'))